# Talks markdown generator for academicpages

Takes a TSV of talks with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `talks.py`. Run either from the `markdown_generator` folder after replacing `talks.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases, rather than Stuart's non-standard TSV format and citation style.

In [1]:
import pandas as pd
import os

## Data format

The TSV needs to have the following columns: title, type, url_slug, venue, date, location, talk_url, description, with a header at the top. Many of these fields can be blank, but the columns must be in the TSV.

- Fields that cannot be blank: `title`, `url_slug`, `date`. All else can be blank. `type` defaults to "Talk" 
- `date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. 
    - The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/talks/YYYY-MM-DD-[url_slug]`
    - The combination of `url_slug` and `date` must be unique, as it will be the basis for your filenames

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [2]:
!cat talks.tsv

title	type	url_slug	venue	date	location	talk_url	description
Real-Time Anomaly Detection from Edge to HPC-Cloud	Talk	bdec2-2018	Big Data and Extreme-Scale Computing 2 Workshop (BDEC2)	2018-11-28	Bloomington, Indiana		
High-Performance AI: A View from Systems and Frameworks	Talk	sc18-2018	Our Deep Learning on Supercomputers workshop of SC18	2018-11-16	Dallas, Texas		
Real-Time Anomaly Detection from Edge to HPC-Cloud	Talk	ccdsc-2018	Workshop on Clusters and Computational Data for Scientific Computing (CCDSC)	2018-09-04	Lyon, France		
High-Performance Big Data Computing with Harp-DAAL	Talk	hpc-2018	HPC 2018 Workshop	2018-07-02	Cetraro, Italy		
Harp-DAAL: A Next Generation Platform for HPC-Cloud	Talk	sc17-2017	SC17	2017-11-15	Denver, Colorado		
Convergence of HPC and Clouds for Data Science	Talk	cetraro-2016	HPC Workshop	2016-07-26	Cetraro, Italy		
Convergence of HPC and Clouds for Data Science	Talk	uiuc-2016	UIUC/NCSA Seminar	2016-03-13	UIUC/NCSA		
Date-enabled Science and Engineering	Ta

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [3]:
talks = pd.read_csv("talks.tsv", sep="\t", header=0)
talks

,title,type,url_slug,venue,date,location,talk_url,description
0,Real-Time Anomaly Detection from Edge to HPC-C...,Talk,bdec2-2018,Big Data and Extreme-Scale Computing 2 Worksho...,2018-11-28,"Bloomington, Indiana",NaN,NaN
1,High-Performance AI: A View from Systems and F...,Talk,sc18-2018,Our Deep Learning on Supercomputers workshop o...,2018-11-16,"Dallas, Texas",NaN,NaN
2,Real-Time Anomaly Detection from Edge to HPC-C...,Talk,ccdsc-2018,Workshop on Clusters and Computational Data fo...,2018-09-04,"Lyon, France",NaN,NaN
3,High-Performance Big Data Computing with Harp-...,Talk,hpc-2018,HPC 2018 Workshop,2018-07-02,"Cetraro, Italy",NaN,NaN
4,Harp-DAAL: A Next Generation Platform for HPC-...,Talk,sc17-2017,SC17,2017-11-15,"Denver, Colorado",NaN,NaN
5,Convergence of HPC and Clouds for Data Science,Talk,cetraro-2016,HPC Workshop,2016-07-26,"Cetraro, Italy",NaN,NaN
6,Convergence of HPC and Clouds for Data Science,Talk,uiuc-2016,UIUC/NCSA Seminar,2016-03-13,UIUC/NCSA,NaN,NaN
7,Date-enabled Science and Engineering,Talk,iu-2015,CS Colloquium at Indiana University,2015-08-28,"Bloomington, Indiana",NaN,NaN
8,Towards HPC-ABDS,Talk,vt-2015,Virginia Tech Seminar,2015-07-09,"Blacksburg, Virginia",NaN,NaN
9,Harp: Collective Communication on Hadoop,Talk,iupui-2014,Computer and Information Science Department Se...,2014-11-07,IUPUI,NaN,NaN


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [4]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    if type(text) is str:
        return "".join(html_escape_table.get(c,c) for c in text)
    else:
        return "False"

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [5]:
loc_dict = {}

for row, item in talks.iterrows():
    
    md_filename = str(item.date) + "-" + item.url_slug + ".md"
    html_filename = str(item.date) + "-" + item.url_slug 
    year = item.date[:4]
    
    md = "---\ntitle: \""   + item.title + '"\n'
    md += "collection: talks" + "\n"
    
    if len(str(item.type)) > 3:
        md += 'type: "' + item.type + '"\n'
    else:
        md += 'type: "Talk"\n'
    
    md += "permalink: /talks/" + html_filename + "\n"
    
    if len(str(item.venue)) > 3:
        md += 'venue: "' + item.venue + '"\n'
        
    if len(str(item.location)) > 3:
        md += "date: " + str(item.date) + "\n"
    
    if len(str(item.location)) > 3:
        md += 'location: "' + str(item.location) + '"\n'
           
    md += "---\n"
    
    
    if len(str(item.talk_url)) > 3:
        md += "\n[More information here](" + item.talk_url + ")\n" 
        
    
    if len(str(item.description)) > 3:
        md += "\n" + html_escape(item.description) + "\n"
        
        
    md_filename = os.path.basename(md_filename)
    #print(md)
    
    with open("../_talks/" + md_filename, 'w') as f:
        f.write(md)

These files are in the talks directory, one directory below where we're working from.

In [6]:
!ls ../_talks

2009-07-31-ndfdicw-2009.md  2012-11-12-mtags-2012.md
2009-09-31-ITsummit-2009.md 2013-03-12-trends-2013.md
2009-10-13-summit09-2009.md 2013-10-10-rutgers-2013.md
2009-10-16-escience-2009.md 2013-11-17-mtags-2013.md
2009-11-16-sc09-2009.md     2014-07-07-hpc-2014.md
2010-05-17-cdn-2010.md      2014-11-07-iupui-2014.md
2010-09-22-fmc-2010.md      2015-07-09-vt-2015.md
2010-12-10-nips-2010.md     2015-08-28-iu-2015.md
2011-03-17-nsf-2011.md      2016-03-13-uiuc-2016.md
2011-04-15-csdc-2011.md     2016-07-26-cetraro-2016.md
2011-07-28-eitc-2011.md     2017-11-15-sc17-2017.md
2011-09-26-peking-2011.md   2018-07-02-hpc-2018.md
2011-09-30-tsinghua-2011.md 2018-09-04-ccdsc-2018.md
2011-11-04-iit-2011.md      2018-11-16-sc18-2018.md
2011-11-12-pdac-2011.md     2018-11-28-bdec2-2018.md
2012-05-07-msft-2012.md     talks.ipynb
2012-06-25-hpc-2012.md      talks.tsv


In [7]:
!cat ../_talks/2013-03-01-tutorial-1.md

cat: ../_talks/2013-03-01-tutorial-1.md: No such file or directory
